### in this set of experiments, we are given a completed ratings matrix

In [1]:
import numpy as np
import math
import time
import matplotlib.pyplot as plt
import random

from scipy import stats

In [2]:
ratingsFile = open("ratingsSummary.csv", "r")

userRatings = dict()
relevantMovies = dict()

for line in ratingsFile:
    data = map(float,line.split(','))
    
    usr = int(data[0])
    mov = int(data[1])
    r = int(data[2])
    
    if usr in userRatings:
        userRatings[usr][mov] = r
    else:
        userRatings[usr] = dict()
        userRatings[usr][mov] = r
        
    relevantMovies[mov] = True
    
print len(userRatings.keys()), len(relevantMovies.keys())

200 1995


### Now, read in the movies.

In [3]:
movies = dict() # a category -> list_of_movies dict stored as integers
movieCats = dict() # movieID -> categories movie is part

# catID returns the index of a category of type string. catName returns the name of the category given its ID.
catID = dict()
catName = dict()

# same as with cat
movieID = dict()
movieName = dict()

In [4]:
allData = open("movies.csv", "r")

# this first line contains header info
allData.readline()

numMovies = 0
numGenres = 0

while True:
    line = allData.readline()
    
    if line == '':
        break
    
    curMovieID = int(line.split(",", 1)[0])
    if curMovieID not in relevantMovies.keys():
        continue
    
    curMovieName = (line.split(",", 1)[1]).rsplit(",", 1)[0]
    curCategories = line.rsplit(",", 1)[1].rsplit("\r")[0].split("|")
    
    
    # update catID, catName, movieID, movieName
    movieID[curMovieName] = curMovieID
    movieName[curMovieID] = curMovieName
    
    for cat in curCategories:
        if not (cat in catID):
            catID[cat] = numGenres
            catName[numGenres] = cat
            
            numGenres = numGenres + 1
            
    movieCats[curMovieID] = curCategories
            
    for cat in curCategories:
        if catID[cat] in movies:
            movies[catID[cat]].append(movieID[curMovieName])
        else:
            movies[catID[cat]] = [movieID[curMovieName]]
    
    numMovies = numMovies + 1

print "we have", numMovies, "movies"

we have 1995 movies


In [5]:
print movies[catID['Horror']][:10]
print movieName[movies[catID['Horror']][0]]
print movieCats[movies[catID['Horror']][0]]
print catID.keys()

[593, 1076, 1200, 1214, 1219, 1258, 1261, 1333, 1340, 1348]
"Silence of the Lambs, The (1991)"
['Crime', 'Horror', 'Thriller']
['Mystery', 'Romance', 'Sci-Fi', 'Musical', 'Film-Noir', 'Crime', 'Drama', 'Fantasy', 'Western', 'Animation', 'War', 'Adventure', 'Horror', 'Action', '(no genres listed)', 'Comedy', 'Documentary', 'Children', 'Thriller', 'IMAX']


In [6]:
print 'We have', len(userRatings.keys()), 'users'
print 'We have', len(movieCats), 'movies'
print 'We have', len(catID.keys()), 'genres'

We have 200 users
We have 1995 movies
We have 20 genres


In [7]:
numMovies = len(movieCats)
numUsers = len(userRatings.keys())

print numMovies, numUsers

1995 200


### Now, complete the ratings matrix

In [8]:
ratingsIncomplete = np.array([[float('NaN') for i in range(numMovies)] for j in range(numUsers)])

movieList = movieName.keys()
userList = userRatings.keys()

for user in userRatings:
    for mov in userRatings[user]:
        line = userList.index(user)
        col = movieList.index(mov)
        
        ratingsIncomplete[line][col] = userRatings[user][mov]

In [9]:
from fancyimpute import SoftImpute

startTime = time.time()

solver = SoftImpute(
    min_value=0.0,
    max_value=5.0)

ratingsComplete = solver.complete(ratingsIncomplete)

print time.time() - startTime

Using TensorFlow backend.


[SoftImpute] Max Singular Value of X_init = 1123.683327
[SoftImpute] Iter 1: observed MAE=0.522688 rank=196
[SoftImpute] Iter 2: observed MAE=0.537585 rank=193
[SoftImpute] Iter 3: observed MAE=0.548181 rank=177
[SoftImpute] Iter 4: observed MAE=0.548157 rank=153
[SoftImpute] Iter 5: observed MAE=0.544339 rank=134
[SoftImpute] Iter 6: observed MAE=0.539971 rank=116
[SoftImpute] Iter 7: observed MAE=0.535780 rank=104
[SoftImpute] Iter 8: observed MAE=0.532160 rank=93
[SoftImpute] Iter 9: observed MAE=0.528867 rank=85
[SoftImpute] Iter 10: observed MAE=0.526145 rank=78
[SoftImpute] Iter 11: observed MAE=0.523777 rank=74
[SoftImpute] Iter 12: observed MAE=0.521712 rank=71
[SoftImpute] Iter 13: observed MAE=0.519792 rank=68
[SoftImpute] Iter 14: observed MAE=0.518061 rank=65
[SoftImpute] Iter 15: observed MAE=0.516570 rank=62
[SoftImpute] Iter 16: observed MAE=0.515245 rank=61
[SoftImpute] Iter 17: observed MAE=0.514135 rank=61
[SoftImpute] Iter 18: observed MAE=0.513141 rank=60
[SoftImput

In [10]:
# now, update the remaining ratings
for user in userList:
    for mov in movieList:
        line = userList.index(user)
        col = movieList.index(mov)
        
        if mov in userRatings[user]:
            assert userRatings[user][mov] == ratingsComplete[line][col], \
            "The initial ratings and completed rating should be equal"
        
        userRatings[user][mov] = ratingsComplete[line][col]

### at this point in time, we should be in the setting from exp_1, but with the filled up ratings matrix

In [11]:
k = 3

In [12]:
from replacementGreedy import replacementGreedy
from greedysum import gsWrapper
from greedymerge import gmWrapper

In [13]:
def computeCostOuter(userIndex, A):
    tot = 0

    curUser = userRatings.keys()[userIndex]

    catA = dict()
    for cat in catID.keys():
        catA[cat] = []
    for mov in A:
        for cat in movieCats[mov]:
            catA[cat].append(mov)

    for cat in catID.keys():
        # now, find the highest rated movie in each category
        highestRated = 0
        for mov in catA[cat]:
            highestRated = max(highestRated, userRatings[curUser][mov])

        tot = tot + highestRated

    return tot

# compute the greedy maximization solution for S for the second stage submodular maximization
def greedyOuter(userIndex, S):
    greedyS = []

    use = [False for s in S]

    for times in range(k):
        # at each step, add the element that gives the greatest marginal gain 

        bestInd = -1
        bestCost = -1

        for ind in range(len(S)):
            if use[ind] == False:
                greedyS.append(S[ind])

                curCost = computeCostOuter(userIndex, greedyS)
                if curCost > bestCost:
                    bestCost = curCost
                    bestInd = ind

                greedyS.pop()

        greedyS.append(S[bestInd])
        use[bestInd] = True

    return computeCostOuter(userIndex, greedyS)

# return the mean, the confidence interval, and the runtime to compute
def computeRatio(S, otherUsers):
    groundSetMean = 0
    for other in otherUsers:
        groundSetMean = groundSetMean + greedyOuter(other, movieName.keys())
    groundSetMean = 1.0 * groundSetMean / len(otherUsers)

    rtS = time.time()
    Sval = []
    for other in otherUsers:
        curVal = greedyOuter(other, S) / groundSetMean
        Sval.append(curVal)
    rtS = time.time() - rtS
    
    print np.mean(Sval)
    
    return np.mean(Sval), \
        stats.norm.interval(0.95, loc=np.mean(Sval), scale=np.std(Sval)/np.sqrt(len(otherUsers))), \
        rtS

In [14]:
#set l
l = 60

rg = replacementGreedy(numMovies, numUsers/2, l, k, \
                            userRatings, movieCats, catID.keys())
rgSs, rgCosts, rgEvals, rgTimes = rg(movieName.keys())

print ''
gs = gsWrapper(numMovies, numUsers/2, l, k, userRatings, movieCats, catID.keys())
gsSs, gsCosts, gsEvals, gsTimes = gs(movieName.keys())

print ''
gm = gmWrapper(numMovies, numUsers/2, l, k, userRatings, movieCats, catID.keys())
gmS, gmCost, gmEvals, gmTime = gm(movieName.keys())

# also, generate a set of random movies, to test the ratios we get against it
randS = random.sample(movieName.keys(), l)

Finished step  2  with cost  5022.77128846 ; number of evals 1198297 ; total runtime 32.9475159645
Finished step  3  with cost  5218.4157217 ; number of evals 2016984 ; total runtime 64.9681239128
Finished step  4  with cost  5361.4479926 ; number of evals 2835488 ; total runtime 94.6756770611
Finished step  5  with cost  5440.04908167 ; number of evals 3640912 ; total runtime 122.418670893
Finished step  6  with cost  5495.30087278 ; number of evals 4445952 ; total runtime 153.39180994
Finished step  7  with cost  5539.53903961 ; number of evals 5249504 ; total runtime 182.200067997
Finished step  8  with cost  5570.50871255 ; number of evals 6051736 ; total runtime 210.353874922
Finished step  9  with cost  5618.90085874 ; number of evals 6854128 ; total runtime 239.19709897
Finished step  10  with cost  5643.74696537 ; number of evals 7655544 ; total runtime 268.164149046
Finished step  11  with cost  5668.19272857 ; number of evals 8456484 ; total runtime 296.713762045
Finished ste

In [15]:
# compute the value ratios and runtime ratios for replacement greedy, greedy sum, greedy merge and the random set
otherUsers = random.sample([x for x in range(numUsers/2, numUsers)], 20)

# compute the runtime for greedyMerge
gmTime = time.time()
val = 0
for other in otherUsers:
    val = val + greedyOuter(other, movieName.keys())
gmTime = time.time() - gmTime
print 'Running greedy merge on the ground set takes', gmTime, 'and returns', val, '\n'

ratioRG = []
runtimeRatioRG = []
for S in rgSs:
    mean, ci, rt = computeRatio(S, otherUsers)
    ratioRG.append((mean, ci))
    runtimeRatioRG.append(rt / gmTime)
print 'Done for replacement greedy\n'
    
ratioGS = []
runtimeRatioGS = []
for S in gsSs:
    mean, ci, rt = computeRatio(S, otherUsers)
    ratioGS.append((mean, ci))
    runtimeRatioGS.append(rt / gmTime)
print 'Done for greedy sum\n'

ratioGM = []
runtimeRatioGM = []
for times in range(len(rgSs)):
    ratioGM.append((1, (1,1)))
    runtimeRatioGM.append(1)
#     mean, ci, rt = computeRatio(movieName.keys(), otherUsers)
#     ratioGM.append((mean, ci))
#     runtimeRatioGM.append(rt / gmTime)
print 'Done for greedy merge\n'

ratioRand = []
runtimeRatioRand = []
for times in range(len(rgSs)):
    mean, ci, rt = computeRatio(randS[:(times+k)], otherUsers)
    ratioRand.append((mean, ci))
    runtimeRatioRand.append(rt / gmTime)
print 'Done for random set\n'

Running greedy merge on the ground set takes 3.62574791908 and returns 1217.32989121 

0.912260370442
0.943476230887
0.945045192416
0.94637036482
0.951307004906
0.952890441697
0.959209889743
0.964005030554
0.964005030554
0.965763995308
0.968431142628
0.971452282062
0.971452282062
0.973300481565
0.973300481565
0.976525965967
0.977432878648
0.981598810803
0.981598810803
0.982892899942
0.984390191752
0.987901154443
0.991386103849
0.991386103849
0.991386103849
0.990131204049
0.992905555285
0.994548495309
0.994548495309
0.994548495309
0.994548495309
0.994548495309
0.99526733558
0.99526733558
0.99526733558
0.99526733558
0.99526733558
0.99526733558
0.99526733558
0.997731745616
0.997939790349
0.997939790349
0.997939790349
0.997939790349
0.997939790349
0.997939790349
0.997939790349
0.997939790349
0.997939790349
0.997939790349
0.997939790349
1.00040420038
1.00040420038
1.00040420038
1.00040420038
1.00060326974
1.00060326974
1.00060326974
Done for replacement greedy

0.912260370442
0.926828201057

In [45]:
plt.clf()

ax = plt.subplot(111)

fs = 17

# http://matplotlib.org/users/text_intro.html
ax.set_xlabel('l', fontsize=fs)
ax.set_ylabel('Ratio', fontsize=fs)

plt.ylim(0.1,1.05)
plt.xlim(k, l)

xticks = [k]
for i in range(10, l + 10, 10):
    xticks.append(i)

ax.set_xticks(xticks)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(fs) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(fs) 

colors = ['r', 'b', 'g', 'y']
labelNames = ['Replacement Greedy', 'Greedy Sum', 'Greedy Merge', 'Random Selection']

for ind in range(len(colors)-1, -1,-1):
    ratios = [ratioRG, ratioGS, ratioGM, ratioRand][ind]
    
    vals = []
    for i in range(len(ratios)):
        vals.append(ratios[i][0])
        
    errTop = []
    errBot = []
    for i in range(len(ratios)):
        errBot.append(vals[i] - ratios[i][1][0])
        errTop.append(ratios[i][1][1] - vals[i])
        
    plt.plot([i for i in range(k,l+1)], vals, c = colors[ind][0], linewidth=2, label = labelNames[ind])
    
    if labelNames[ind] != 'Greedy Merge':
        plt.errorbar([i for i in range(k,l+1)], vals, yerr = [errBot, errTop], fmt = colors[ind])
    
# http://matplotlib.org/1.3.0/examples/pylab_examples/legend_demo.html
legend = ax.legend(loc='lower right')

# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize(fs)

# plt.show()
plt.savefig("../../writeup/images/movielens-sublinear-exp2-ratio")

plt.close()

In [47]:
for curL in [7,27,57]:
    """
    ========
    Barchart
    ========

    A bar plot with errorbars and height labels on individual bars
    """
    import numpy as np
    import matplotlib.pyplot as plt

    fs = 17
    lfs = 15

    N = 4

    means = (1.0, ratioRG[curL][0], ratioGS[curL][0], ratioRand[curL][0])
    runtimes = (1.0, runtimeRatioRG[curL], runtimeRatioGS[curL], runtimeRatioRand[curL])

    ind = np.arange(N)  # the x locations for the groups
    width = 0.3       # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(ind, means, width, color='r')
    rects2 = ax.bar(ind + width, runtimes, width, color='y')

    # add some text for labels, title and axes ticks
    ax.set_ylabel('Performance', fontsize = lfs)
    # ax.set_title('Scores by group and gender')
    ax.set_xticks(ind + width / 2)
    ax.set_xticklabels(('Greedy\nMerge', 'Replacement\nGreedy', 'Greedy\nSum', 'Random'), fontsize = lfs)
    #                    rotation='vertical')

    ax.legend((rects1[0], rects2[0]), ('Objective Value', 'Runtime'))

    plt.ylim(0,1.3)

    def autolabel(rects):
        """
        Attach a text label above each bar displaying its height
        """

        for rect in rects:
            height = rect.get_height()
            if rect != rects[0] > 0:
                ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                        '~%.f%%   ' % (100 * height),
                        ha='center', va='bottom')
            else:
                ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                        '  %.f%%' % (100 * height),
                        ha='center', va='bottom')

    def autolabel2(rects):
        """
        Attach a text label above each bar displaying its height
        """
        for rect in rects:
            height = rect.get_height()
            if rect != rects[0] > 0:
                ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                        '~%.f%%' % (100 * height),
                        ha='left', va='bottom')
            else:
                ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                        '%.f%%' % (100 * height),
                        ha='left', va='bottom')

    autolabel(rects1)
    autolabel2(rects2)

    # plt.show()

    plt.savefig("../../writeup/images/movielens-sublinear-exp2-barplot-" + str(curL + k))

In [56]:
for curL in [7,27,57]:
    """
    ========
    Barchart
    ========

    A bar plot with errorbars and height labels on individual bars
    """
    import numpy as np
    import matplotlib.pyplot as plt

    fs = 17
    lfs = 15

    N = 4

    eps = 0.005
    means = (max(1 - ratioRG[curL][0], eps), 1 - ratioGS[curL][0], 1 - ratioRand[curL][0], eps)
    runtimes = (max(runtimeRatioRG[curL], eps), runtimeRatioGS[curL], runtimeRatioRand[curL], 1.0)
    
    print means
    
    ind = np.arange(N)  # the x locations for the groups
    width = 0.3       # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(ind, means, width, color='r')
    rects2 = ax.bar(ind + width, runtimes, width, color='y')

    # add some text for labels, title and axes ticks
    ax.set_ylabel('Performance', fontsize = lfs)
    # ax.set_title('Scores by group and gender')
    ax.set_xticks(ind + width / 2)
    ax.set_xticklabels(('Replacement\nGreedy', 'Greedy\nSum', 'Random', 'Greedy\nMerge'), fontsize = lfs)
    #                    rotation='vertical')

    ax.legend((rects1[0], rects2[0]), ('Loss', 'Runtime'), loc='upper left')

    plt.ylim(0,1.2)

    def autolabel(rects):
        """
        Attach a text label above each bar displaying its height
        """

        for rect in rects:
            height = rect.get_height()
            if rect != rects[0] > 0:
                ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                        '~%.f%%   ' % (100 * height),
                        ha='center', va='bottom')
            else:
                ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                        '  %.f%%' % (100 * height),
                        ha='center', va='bottom')

    def autolabel2(rects):
        """
        Attach a text label above each bar displaying its height
        """
        for rect in rects:
            height = rect.get_height()
            if rect != rects[0] > 0:
                ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                        '~%.f%%' % (100 * height),
                        ha='left', va='bottom')
            else:
                ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                        '%.f%%' % (100 * height),
                        ha='left', va='bottom')

    autolabel(rects1)
    autolabel2(rects2)

    # plt.show()

    plt.savefig("../../writeup/images/movielens-sublinear-exp2-barplot-loss-" + str(curL + k))

(0.035994969446008218, 0.040269277016489413, 0.41530581350580065, 0.005)
(0.0054515046913354848, 0.029032632670152658, 0.3570825581880559, 0.005)
(0.005, 0.021524669008650021, 0.23802531214219902, 0.005)


In [40]:
# store locally
filename = open('../data/movielens-exp2.txt', 'w')

def writeInfo(text, values, rt):
    print>>filename, text
    for ind in range(len(values)):
        print>>filename, str(values[ind][0]), str(values[ind][1][0]), str(values[ind][1][1]), str(rt[ind])

writeInfo('RG',ratioRG, runtimeRatioRG)
writeInfo('GS',ratioGS, runtimeRatioGS)
writeInfo('GM',ratioGM, runtimeRatioGM)
writeInfo('Rand',ratioRand, runtimeRatioRand)

In [76]:
def computeCostOuter2(userIndex, A):
    tot = 0

    curUser = userRatings.keys()[userIndex]

    catA = dict()
    for cat in catID.keys():
        catA[cat] = []
    for mov in A:
        for cat in movieCats[mov]:
            catA[cat].append(mov)

    for cat in catID.keys():
        # now, find the highest rated movie in each category
        highestRated = 0
        for mov in catA[cat]:
            highestRated = max(highestRated, userRatings[curUser][mov])

        tot = tot + highestRated

    return tot

def greedyOuter2(userIndex, S):
    greedyS = []

    use = [False for s in S]

    for times in range(k):
        # at each step, add the element that gives the greatest marginal gain 

        bestInd = -1
        bestCost = -1

        for ind in range(len(S)):
            if use[ind] == False:
                greedyS.append(S[ind])

                curCost = computeCostOuter2(userIndex, greedyS)
                if curCost > bestCost:
                    bestCost = curCost
                    bestInd = ind

                greedyS.pop()

        greedyS.append(S[bestInd])
        use[bestInd] = True

    return greedyS, computeCostOuter2(userIndex, greedyS)

In [93]:
# find the 3 movies that three users like

usr = otherUsers[0]
print usr
for mov in greedyOuter2(usr, rgSs[57])[0]:
    print movieName[mov]
    
usr = otherUsers[0]
print usr
for mov in greedyOuter2(usr, gsSs[57])[0]:
    print movieName[mov]
    
usr = otherUsers[0]
print usr
for mov in greedyOuter2(usr, randS)[0]:
    print movieName[mov]

 171
Inception (2010)
"Monsters, Inc. (2001)"
Waltz with Bashir (Vals im Bashir) (2008)
171
Inception (2010)
Shrek (2001)
Night and Fog (Nuit et brouillard) (1955)
171
"Princess Bride, The (1987)"
Drive (2011)
Anvil! The Story of Anvil (2008)
60 60
